### Data 620 - Project 1
### Bikram Barua and Leticia Salazar
### February 19, 2023

#### Task: 

For your first project, you are asked to:
1. Identify and load a network dataset that has some categorical information available for each node.
2. For each of the nodes in the dataset, calculate degree centrality and eigenvector centrality.
3. Compare your centrality measures across your categorical groups.

For example, using the dataset from a study of romantic relationships among high school students in Columbus, Ohio [http://researchnews.osu.edu/archive/chains.htm], you might want to determine if there are any differences in your calculated centrality measures between the two sexes.  You might use a t-test or another statistical measure to support your findings.

#### Data source:

The dataset selected for project 1 was obtained from Network Repository consisting of people (nodes) and their friendship ties (edges) on Facebook. The data is available in a matrix market, typically a sparse format used to represent a matrix (.mtx file). With these types of files, the first line contains a header with information about the matrix, dimensions, and symmetry of the matrix. The following lines are non-zero entries of the matrix in row-major order, with each following line containing the row index, column index, and a value of a single non-zero entry.

Below are some specs from this dataset provided by the repository:

* Nodes	-> 18.7K
* Edges	-> 790.8K
* Density -> 0.00454239
* Maximum degree -> 3.2K
* Minimum degree -> 1
* Average degree -> 84
* Assortativity  -> 0.01805
* Number of triangles -> 18.3M
* Average number of triangles -> 982
* Maximum number of triangles -> 77.2K
* Average clustering coefficient -> 0.219051
* Fraction of closed triangles -> 0.135615
* Maximum k-core -> 85
* Lower bound of Maximum Clique -> 13

#### Project Objectives: 

We will be focusing to answer the following while we explore, analyze and create visualizations for our data.

* Which centrality measure would be the most relevant in the dataset?
* What would these centrality measures help us to predict?
* What statistical tests can we apply to identify the best fit centrality measure?

#### Load libraries:

Below are the libraries used

In [2]:
# data manipulation
import pandas as pd
import numpy as np
import scipy.io #for mtx files

# data visualization
import networkx as nx
import matplotlib.pyplot as plt

#### Load the data:
Data exploration of the data is performed below to view the size of the datasets we will be working with.

In [18]:
#M = scipy.io.mmread('/Users/letisalba/Desktop/Data-620/Week-4/socfb-Cornell5.mtx')
df = mmread(socfb-Cornell5.mtx)

NameError: name 'socfb' is not defined

In [6]:
df = pd.read_csv('https://raw.githubusercontent.com/letisalba/Data-620/master/Week-4/fb-cornell5.csv')
print(df.head(10))

   18660  18660.1  790777
0    184        1     NaN
1    344        1     NaN
2    635        1     NaN
3    867        1     NaN
4   1296        1     NaN
5   1358        1     NaN
6   1425        1     NaN
7   1501        1     NaN
8   1505        1     NaN
9   1845        1     NaN


In [4]:
df.shape

(790776, 3)

In [5]:
df.describe

<bound method NDFrame.describe of         18660  18660.1  790777
0         184        1     NaN
1         344        1     NaN
2         635        1     NaN
3         867        1     NaN
4        1296        1     NaN
...       ...      ...     ...
790771  18655    18600     NaN
790772  18608    18603     NaN
790773  18612    18603     NaN
790774  18638    18610     NaN
790775  18653    18612     NaN

[790776 rows x 3 columns]>

#### References:

* Rossi, R. A., & Ahmed, N. K. (2015). The Network Data Repository with Interactive Graph Analytics and Visualization. Network Data Repository. Retrieved February 16, 2023, from https://networkrepository.com 
* AOMAR, A. A. I. T. (2020, August 2). Notes on graph theory - centrality measures. Medium. Retrieved February 17, 2023, from https://towardsdatascience.com/notes-on-graph-theory-centrality-measurements-e37d2e49550a 
* Disney, A. (2022, October 27). Social network analysis: Understanding centrality measures. Cambridge Intelligence. Retrieved February 18, 2023, from https://cambridge-intelligence.com/keylines-faqs-social-network-analysis/ 
* ArijitGayen@ArijitGayen. (2019, November 21). Network centrality measures in a graph using Networkx: Python. GeeksforGeeks. Retrieved February 18, 2023, from https://www.geeksforgeeks.org/network-centrality-measures-in-a-graph-using-networkx-python/ 